In [1]:
import pandas as pd
import datetime
from datetime import datetime
import numpy as np

In [2]:
data_df=pd.read_csv('Recruitment_pipeline_coded.csv')

C:\Users\ledwa\AppData\Local\Temp\ipykernel_29780\2042880330.py:1: DtypeWarning: Columns (26,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df=pd.read_csv('Recruitment_pipeline_coded.csv')


In [3]:
data_df=data_df.drop(labels=['Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29'], axis = 1)
#data_df.dropna(axis=0)

In [51]:
#data_df=data_df[data_df['sex'].notna()]
#data_df=data_df[data_df['initial_contact_method'].notna()]
#data_df=data_df[data_df['visit_outcome'].notna()]
#data_df=data_df[data_df['study_id']].drop_duplicates()
#data_df=data_df.set_index('study_id')
#df=data_df.groupby(by=['study_id'], axis=1, as_index='True')
#data_df
#data_df.to_csv('test.csv')
#data_df['study_id'].value_counts()

### calculating the number of times that the participant was schedualed to show up
data_df['freq']=data_df.groupby('study_id')['study_id'].transform('count') - 1

### condesing all rows that are duplicates on the study_id into one. WARNING this is not perfect and we are losing some data in the visit_planned_date through the visit_outcome data. Right now it just picks either the first or last data that came up in those columns when combinding them
condensed_data=data_df.groupby(data_df['study_id']).aggregate({'study_id': 'first', 'sex': 'first', 'birthyear': 'first', 'initial_date': 'first', 'initial_contact_method': 'first', 'fu_dt_1':  'first', 'fu_method_1': 'first', 'fu_outcome_1': 'first', 'fu_dt_2': 'first'
,'fu_method_2': 'first', 'fu_outcome_2': 'first', 'fu_dt_3':'first', 'fu_method_3': 'first', 'fu_outcome_3': 'first', 'fu_dt_4':'first', 'fu_method_4': 'first', 'fu_outcome_4': 'first',  'fu_dt_5':'first', 'fu_method_5': 'first', 'fu_outcome_5': 'first', 'visit_planned_date': 'first'
, 'visit_reminder_date': 'first', 'actual_visit_date':'first', 'visit_weekday':'first', 'coded_location':'first', 'visit_outcome':'last', 'freq':'first'})

### droping data where visit outcome is na
condensed_data=condensed_data[condensed_data['visit_outcome'].notna()]

### making a data frame of just the contact attempts before the appointment
count_stuff=condensed_data[['fu_dt_1','fu_dt_2', 'fu_dt_3', 'fu_dt_4', 'fu_dt_5']]

### calculating the number of attempts made to contact the participant
condensed_data['contact_count']= 5- count_stuff.isnull().sum(axis=1)

### converting the birth year to a date
condensed_data['birthyear']=pd.to_datetime(condensed_data['birthyear'], format='%Y', exact=False)

### converting the actual_visit_date to a year
condensed_data['actual_visit_date']=pd.to_datetime(condensed_data['actual_visit_date'], format='%Y', exact=False)

### Calculating the age of the person at the time of their visit.... its stuck in days and not years right now :/
#condensed_data['Age_at_visit']=(condensed_data['actual_visit_date']- condensed_data['birthyear'])

condensed_data['birthyear'] = condensed_data.birthyear.dt.year

#condensed_data.age = condensed_data.age.fillna(-1)


# condensed_data['new_age'] = condensed_data['age'].apply(np.int64)

condensed_data['year'] = condensed_data.actual_visit_date.dt.year
condensed_data['age']= condensed_data['year'] - condensed_data['birthyear']
# condensed_data['time_fu_dt_2'] = condensed_data['fu_dt_2'].time()

#condensed_data['new_age'] = condensed_data['age'].apply(np.int64)

condensed_data["date_time_fu_dt_2"] = pd.to_datetime(condensed_data["fu_dt_2"])

#for d in condensed_data['date_time_fu_dt_2']:
 # condensed_data['date_fu_dt_2'] = d.date()
  #condensed_data['time_fu_dt_2'] = d.time()

condensed_data['date_fu_dt_2'] = pd.to_datetime(condensed_data['date_time_fu_dt_2']).dt.date
condensed_data['time_fu_dt_2'] = pd.to_datetime(condensed_data['date_time_fu_dt_2']).dt.time

condensed_data['age'] = condensed_data['age'].astype('Int16')



study_id,index,0001b470ffdea9daf4e3e25a9cfb93d0,0003a8af34dd38929b6d0dff83a281e5,00042cd8cfa979e5cbf9aa79a01ec23d,0014065de00658be8463cc209aae5bc1,001e2c5b018d498cd8498e2b9d0ce0c7,003700014c86a1fa205fa052e661338f,0039979ebfff72785e59b828211f41c3,003a787ae052b1441d6d47f2d606b68d,003e85653a119fd249d2dc3a0137ef40,...,ffd8350b21c14115f92fb680fc52ea5b,ffdb7ef1abb21488664aaac82e01f5f7,ffdc4b84b6f0bfabf54e98d6e15f4efd,ffe05bb78fee90c5c6ed06610a7d8635,ffe1c73c776265a2351dca5f75ed2319,ffec65b7a5fd2d36cd56e0b249322d1f,ffeeb12c35a69dcdd34c4b2db6951789,ffef6be3cbb0c80342dba653133169df,fffdb5ffd34614801b0eb102ad4527bd,ffff49a8692e072fcb5df471ac7bd82a
0,fu_dt_5,5/17/2022 14:56,None,None,3/22/2022 16:54,None,None,None,None,11/10/2021 14:51,...,None,None,None,None,None,None,None,None,None,None
1,fu_dt_4,4/26/2022 15:46,None,5/5/2022 16:17,3/16/2022 13:26,None,None,None,None,8/14/2019 12:39,...,None,None,None,3/15/2019 12:02,None,None,None,None,None,None
2,fu_dt_3,4/25/2022 12:22,None,4/13/2022 11:40,2/25/2022 11:58,None,5/13/2019 10:32,9/19/2022 10:25,None,8/12/2019 17:09,...,None,9/27/2018 11:03,None,3/12/2019 13:12,None,None,12/8/2021 17:44,None,None,9/21/2022 17:21
3,fu_dt_2,4/8/2022 14:38,None,4/8/2022 12:25,2/24/2022 17:13,None,4/5/2019 18:57,9/16/2022 12:21,None,7/29/2019 12:34,...,None,9/27/2018 10:48,None,3/5/2019 13:21,None,None,10/23/2019 13:34,None,None,3/4/2022 14:49
4,fu_dt_1,3/9/2021 15:26,10/16/2019 15:11,7/8/2021 12:40,1/21/2020 17:50,None,3/22/2019 9:50,9/13/2022 10:01,2/5/2020 16:16,7/25/2019 13:21,...,1/21/2021 13:33,9/25/2018 15:06,9/5/2019 10:01,3/5/2019 11:14,1/15/2020 13:40,1/13/2020 9:54,10/21/2019 15:31,None,4/26/2019 9:00,3/5/2021 15:26


In [73]:
count_stuff=condensed_data[['fu_dt_5','fu_dt_4', 'fu_dt_3', 'fu_dt_2', 'fu_dt_1']]

#something=count_stuff.T
#column_list=something.columns.tolist()
#something=something.reset_index()
#column_list
#hope=pd.DataFrame()
#something.fillna("",inplace=True)
#for item in column_list:
    #test=something[item].apply(' '.join).reset_index()
new4=count_stuff['fu_dt_4']
new3=count_stuff['fu_dt_3']
new2=count_stuff['fu_dt_2']
new1=count_stuff['fu_dt_1']



count_stuff["combined_date"]= count_stuff['fu_dt_5']
count_stuff["combined_date"].fillna('',inplace=True)
count_stuff["combined_date"]=count_stuff["combined_date"].str.cat(new4, sep =" ")
count_stuff["combined_date"].fillna('',inplace=True)
count_stuff["combined_date"]=count_stuff["combined_date"].str.cat(new3, sep =" ")
count_stuff["combined_date"].fillna('',inplace=True)
count_stuff["combined_date"]=count_stuff["combined_date"].str.cat(new2, sep =" ")
count_stuff["combined_date"].fillna('',inplace=True)
count_stuff["combined_date"]=count_stuff["combined_date"].str.cat(new1, sep =" ")
count_stuff["combined_date"].fillna('0:00',inplace=True)
condensed_data['combined_date']=count_stuff['combined_date']
condensed_data

C:\Users\ledwa\AppData\Local\Temp\ipykernel_29780\2612907746.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  count_stuff["combined_date"]= count_stuff['fu_dt_5']
C:\Users\ledwa\AppData\Local\Temp\ipykernel_29780\2612907746.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  count_stuff["combined_date"].fillna('',inplace=True)
C:\Users\ledwa\AppData\Local\Temp\ipykernel_29780\2612907746.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,study_id,sex,birthyear,initial_date,initial_contact_method,fu_dt_1,fu_method_1,fu_outcome_1,fu_dt_2,fu_method_2,...,coded_location,visit_outcome,freq,contact_count,year,age,date_time_fu_dt_2,date_fu_dt_2,time_fu_dt_2,combined_date
study_id,,,,,,,,,,,,,,,,,,,,,
0001b470ffdea9daf4e3e25a9cfb93d0,0001b470ffdea9daf4e3e25a9cfb93d0,2.0,1980.0,2/28/2021,email,3/9/2021 15:26,phone,None,4/8/2022 14:38,phone,...,7.0,no_show,2,5,2021,41,2022-04-08 14:38:00,2022-04-08,14:38:00,5/17/2022 14:56 4/26/2022 15:46 4/25/2022 12:2...
0003a8af34dd38929b6d0dff83a281e5,0003a8af34dd38929b6d0dff83a281e5,NaN,NaN,10/16/2019,self_referred,10/16/2019 15:11,phone,None,None,None,...,2.0,completed,1,1,2019,<NA>,NaT,NaT,NaT,10/16/2019 15:11
00042cd8cfa979e5cbf9aa79a01ec23d,00042cd8cfa979e5cbf9aa79a01ec23d,2.0,1947.0,6/30/2021,email,7/8/2021 12:40,phone,crc_followup,4/8/2022 12:25,phone,...,1.0,no_show,3,4,2022,75,2022-04-08 12:25:00,2022-04-08,12:25:00,5/5/2022 16:17 4/13/2022 11:40 4/8/2022 12:25...
0014065de00658be8463cc209aae5bc1,0014065de00658be8463cc209aae5bc1,2.0,1961.0,1/17/2020,email,1/21/2020 17:50,phone,None,2/24/2022 17:13,phone,...,7.0,canceled,3,5,2020,59,2022-02-24 17:13:00,2022-02-24,17:13:00,3/22/2022 16:54 3/16/2022 13:26 2/25/2022 11:5...
001e2c5b018d498cd8498e2b9d0ce0c7,001e2c5b018d498cd8498e2b9d0ce0c7,2.0,1958.0,3/13/2020,email,None,None,None,None,None,...,2.0,rescheduled,2,0,2022,64,NaT,NaT,NaT,0:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffec65b7a5fd2d36cd56e0b249322d1f,ffec65b7a5fd2d36cd56e0b249322d1f,NaN,1970.0,1/13/2020,in_person,1/13/2020 9:54,phone,None,None,None,...,2.0,completed,1,1,2020,50,NaT,NaT,NaT,1/13/2020 9:54
ffeeb12c35a69dcdd34c4b2db6951789,ffeeb12c35a69dcdd34c4b2db6951789,2.0,1955.0,9/22/2019,email,10/21/2019 15:31,vm,None,10/23/2019 13:34,phone,...,5.0,reschedule_me_later,1,3,2019,64,2019-10-23 13:34:00,2019-10-23,13:34:00,12/8/2021 17:44 10/23/2019 13:34 10/21/2019 1...
ffef6be3cbb0c80342dba653133169df,ffef6be3cbb0c80342dba653133169df,1.0,1933.0,9/28/2019,email,None,None,None,None,None,...,7.0,canceled,1,0,2019,86,NaT,NaT,NaT,0:00


In [117]:
condensed_data.dtypes

study_id                          object
sex                              float64
birthyear                        float64
initial_date                      object
initial_contact_method            object
fu_dt_1                           object
fu_method_1                       object
fu_outcome_1                      object
fu_dt_2                           object
fu_method_2                       object
fu_outcome_2                      object
fu_dt_3                           object
fu_method_3                       object
fu_outcome_3                      object
fu_dt_4                           object
fu_method_4                       object
fu_outcome_4                      object
fu_dt_5                           object
fu_method_5                       object
fu_outcome_5                      object
visit_planned_date                object
visit_reminder_date               object
actual_visit_date         datetime64[ns]
visit_weekday                     object
coded_location  

In [6]:
condensed_data.to_csv('test1.csv')